# Mosaic the individual 1km^2 tiles into a single stack of large rasters for inference and analysis
* Main thing to watch: correctly interpolating gaps at tile edges without wiping out useful nodata mask
* This will be unnecessary when orthoimages correctly projected onto ASP DEM (making a full stack without referring to tiles)

In [12]:
import rioxarray
import rasterio.fill

In [3]:
%pwd

'/mnt/1.0_TB_VOLUME/sethv/resdepth_all/deep-elevation-refinement/ResDepth/torchgeo_experiments'

In [4]:
!mkdir -p mosaic

In [5]:
%cd mosaic

/mnt/1.0_TB_VOLUME/sethv/resdepth_all/deep-elevation-refinement/ResDepth/torchgeo_experiments/mosaic


In [6]:
NODATA_VALUE_FOR_EDGES = -88888888


In [39]:
ddir="/mnt/1.0_TB_VOLUME/sethv/shashank_data/TRAIN_tile_stack_baker_128_global_coreg /mnt/1.0_TB_VOLUME/sethv/shashank_data/VALIDATION_tile_stack_baker_128_global_coreg/"
patterns = {
    "nadir_ortho": "1020010042D39D00.r100_ortho_1.0m_ba.tif",
    "offnadir_ortho": "1020010043455300.r100_ortho_1.0m_ba.tif",
    "initial_dem_root": "WV01_20150911_1020010042D39D00_1020010043455300_aligned_crop_1.0m-DEM_holes_filled.tif",
    "initial_dem_unfilled_root": "WV01_20150911_1020010042D39D00_1020010043455300_aligned_crop_1.0m-DEM.tif",
    "target_root": "USGS_LPC_WA_MtBaker_2015_*_LAS_2017_32610_first_filt_v1.3_1.0m-DEM_holes_filled.tif",
    "triangulation_error_root": "WV01_20150911_1020010042D39D00_1020010043455300_aligned_crop_1.0m-IntersectionErr.tif"
}

for name, pattern in patterns.items():
    !find $ddir -path "*files_to_zip/$pattern" | wc -l #> nadir_orthos_128.txt
    # want to find the seams and interpolate those, temporary fix until lidar + stereo stack is aligned properly
    # -ul_lr 582000 5411000 591000 5395000 -tap did not work for getting orthoimage missing pixel fixed
    out_fn = f"mosaic_full128_{pattern}"
    !gdal_merge.py -o "$out_fn" -init -88888888  `find $ddir -path "*files_to_zip/$pattern"`
    if "ortho" in name: # fix the missing pixel (Baker specific one-off bug)
        fixed_fn = f"fixed_{out_fn}"
        !gdal_translate -projwin 583000 5411000 591000 5395000 "$out_fn" "$fixed_fn"
        !mv $fixed_fn $out_fn
    
# gdal_merge.py -o mosaic_full128_$pattern_nadir_ortho `
# !find $ddir -path "*files_to_zip/$pattern_nadir_ortho" > nadir_orthos_128.txt

128
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7999, 15999
Warning 1: Computed -srcwin 0 -1 8000 16000 falls partially outside raster extent. Going on however.
0...10...20...30...40...50...60...70...80...90...100 - done.
128
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7999, 15999
Warning 1: Computed -srcwin 0 -1 8000 16000 falls partially outside raster extent. Going on however.
0...10...20...30...40...50...60...70...80...90...100 - done.
128
0...10...20...30...40...50...60...70...80...90...100 - done.
128
0...10...20...30...40...50...60...70...80...90...100 - done.
128
0...10...20...30...40...50...60...70...80...90...100 - done.
128
0...10...20...30...40...50...60...70...80...90...100 - done.


In [ ]:
!gdal_translate 

In [16]:
!pwd

/mnt/1.0_TB_VOLUME/sethv/resdepth_all/deep-elevation-refinement/ResDepth/torchgeo_experiments/mosaic


In [38]:
!gdalinfo fixed_mosaic_full128_1020010042D39D00.r100_ortho_1.0m_ba.tif

Driver: GTiff/GeoTIFF
Files: fixed_mosaic_full128_1020010042D39D00.r100_ortho_1.0m_ba.tif
Size is 9000, 16000
Coordinate System is:
PROJCRS["WGS 84 / UTM zone 10N",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["UTM zone 10N",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",-123,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
            ID["EPSG",8805]],
        PARAMETER["False easting",500000,
            LENGTHUNIT["metre

In [42]:
!gdalinfo files_to_zip/1020010042D39D00.r100_ortho_1.0m_ba.tif

Driver: GTiff/GeoTIFF
Files: files_to_zip/1020010042D39D00.r100_ortho_1.0m_ba.tif
Size is 8000, 16000
Coordinate System is:
PROJCRS["WGS 84 / UTM zone 10N",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["UTM zone 10N",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",-123,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
            ID["EPSG",8805]],
        PARAMETER["False easting",500000,
            LENGTHUNIT["metre",1],
  

In [8]:
!mkdir -p files_to_zip

In [44]:
for name, fn in patterns.items():
    print(f"Interpolating {name}")
    fp = f"mosaic_full128_{fn}"
    
#     with rasterio.open(fn) as arr:
#         out = rasterio.fill.fillnodata(arr, arr != NODATA_VALUE_FOR_EDGES)
#     with rasterio.open(fp, "w") as out_fn:
#         out_fn.write(out)

    rxr_ds = rioxarray.open_rasterio(fp) # without masked=True, only want to modify missing edges between tiles
    # rxr_ds.rio.set_nodata(NODATA_VALUE_FOR_EDGES) # intersection error holes are 0s so these holes *won't* be interpolated
    # print("Interpolating with rioxarray")
    # rxr_ds.rio.interpolate_na("nearest")
    print("Filling edges with rasterio fillnodata")
    rxr_ds.data = rasterio.fill.fillnodata(rxr_ds, (rxr_ds != 0) & (rxr_ds != NODATA_VALUE_FOR_EDGES))#, smoothing_iterations=1)
    print("Writing to file")
    rxr_ds.rio.to_raster("files_to_zip/"+fn)
    print(f"Done with {fp}")
    # break

Interpolating nadir_ortho
Filling edges with rasterio fillnodata
Writing to file
Done with mosaic_full128_1020010042D39D00.r100_ortho_1.0m_ba.tif
Interpolating offnadir_ortho
Filling edges with rasterio fillnodata
Writing to file
Done with mosaic_full128_1020010043455300.r100_ortho_1.0m_ba.tif
Interpolating initial_dem_root
Filling edges with rasterio fillnodata
Writing to file
Done with mosaic_full128_WV01_20150911_1020010042D39D00_1020010043455300_aligned_crop_1.0m-DEM_holes_filled.tif
Interpolating initial_dem_unfilled_root
Filling edges with rasterio fillnodata
Writing to file
Done with mosaic_full128_WV01_20150911_1020010042D39D00_1020010043455300_aligned_crop_1.0m-DEM.tif
Interpolating target_root
Filling edges with rasterio fillnodata
Writing to file
Done with mosaic_full128_USGS_LPC_WA_MtBaker_2015_*_LAS_2017_32610_first_filt_v1.3_1.0m-DEM_holes_filled.tif
Interpolating triangulation_error_root
Filling edges with rasterio fillnodata
Writing to file
Done with mosaic_full128_WV01